In [89]:
from pyscf import gto,scf
import numpy as np

## Fractional charge molecules

In [3]:
mol_NN=gto.M(atom= "N 0 0 0; N 0 0 2.1",unit="Bohr", basis="sto-3g")

In [32]:
#molecule with fractional charges 
%load_ext autoreload
%autoreload 2
from FcMole import FcM, FcM_like

In [12]:
#first method using FcM
fmol1=FcM(fcs=[1,1],atom= "N 0 0 0; N 0 0 2.1",unit="Bohr", basis="sto-3g") #fcs is a list with fcs for every atom

In [14]:
fmol1.charge,fmol1.nelec,fmol1.atom_charges()

(2, (7, 7), array([8., 8.]))

In [49]:
#second method using FcM_like fcs can also be parsed via a doble list [[idx_n,..],[fc_n,..]]
]fmol2=FcM_like(mol_NN,fcs=[[0],[.3]])

In [50]:
fmol2.charge,fmol2.nelec,fmol2.atom_charges()

(0.3, (7, 7), array([7.3, 7. ]))

In [66]:
# you can also add charge to the initial molecule, the fcs don't affect the number of electrons
fmol3=FcM(fcs=[[0,1],[1,1]],atom= "N 0 0 0; N 0 0 2.1",unit="Bohr", basis="sto-3g",spin=1,charge=-1)

In [68]:
fmol3.charge,fmol3.nelec,fmol3.atom_charges()

(1, (8, 7), array([8., 8.]))

In [51]:
mf1,mf2,mf3=scf.RHF(fmol1),scf.RHF(fmol2),scf.RHF(fmol3)
mf1.scf(dm0=mf1.init_guess_by_1e())
mf2.scf(dm0=mf2.init_guess_by_1e())
mf3.scf(dm0=mf3.init_guess_by_1e())

converged SCF energy = -112.220627534812
converged SCF energy = -112.220627534812
converged SCF energy = -143.609288387384


-143.60928838738442

In [63]:
mf1.mo_occ,sum(mf1.mo_occ),mf2.mo_occ,sum(mf2.mo_occ) #checking occupations

(array([2., 2., 2., 2., 2., 2., 2., 0., 0., 0.]),
 14.0,
 array([2., 2., 2., 2., 2., 2., 2., 0., 0., 0.]),
 14.0)

In [64]:
mf3.mo_occ,sum(mf3.mo_occ)

(NPArrayWithTag([2., 2., 2., 2., 2., 2., 2., 0., 1., 0.]), 15.0)

## Using the AP_class

In [70]:
from AP_class import APDFT_perturbator as AP

In [73]:
mf_nn=scf.RHF(mol_NN)
#need to run the scf first
mf_nn.scf()
ap_nn=AP(mf_nn,sites=[0,1])

converged SCF energy = -107.49885049543


In [76]:
#build the alchemical gradient dE/dZ_i
ap_nn.build_gradient(0,1)

array([-17.96252176, -17.96252176])

In [79]:
#build the alchemical hessian d**2E/dZ_i/dZ_j
ap_nn.build_hessian(0,1)

array([[-0.56261427,  0.95186321],
       [ 0.95186321, -0.56261427]])

In [87]:
ap_nn.build_cubic_hessian(0,1)

array([[[-0.05028813, -0.00297524],
        [-0.00297524, -0.00297524]],

       [[-0.00297524, -0.00297524],
        [-0.00297524, -0.05028813]]])

In [95]:
#you can retreve the derivatives calling them 
ap_nn.gradient,ap_nn.hessian,ap_nn.cubic_hessian

(array([-17.96252176, -17.96252176]), array([[-0.56261427,  0.95186321],
        [ 0.95186321, -0.56261427]]), array([[[-0.05028813, -0.00297524],
         [-0.00297524, -0.00297524]],
 
        [[-0.00297524, -0.00297524],
         [-0.00297524, -0.05028813]]]))

In [ ]:
#after building hessian and gradient you can call APDFTn predictions of a numpy array that 
# constitutes the perturbation

In [90]:
ap_nn.APDFT3(np.asarray([-1,1])) # to CO

-109.01332797130448

In [91]:
ap_nn.APDFT3(np.asarray([1,-1])) # to OC

-109.01332797130448

In [96]:
ap_nn.APDFT2(np.asarray([-1,1]))  # due to symmetry APDFT2 has the same energy as APDFT3

-109.01332797130448

In [98]:
ap_nn.APDFT1(np.asarray([1,1])) # going to OO++

-143.42389401003226

etc.etc.


##  Things to be improved 
1) handling of sites <br>
2) use symmetry to reduce the number of calculation required is trick because the AO coefficients needs to be rotated <br> 
3) Mixing with finite difference to get higher order ? <br>
4) DFT implementation 
